In [1]:
#!pip install crewai crewai_tools langchain_community sentence-transformers langchain-groq langchain_huggingface --quiet

In [19]:
#!pip install langchain

In [31]:
# set GROQ_API_KEY and TAVILY_API_KEY
from dotenv import load_dotenv
load_dotenv()


True

In [32]:
from langchain.agents import tool
from crewai_tools import PDFSearchTool
from crewai import Agent, Crew, Task,Process
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI


In [33]:

"""
llm =ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    #model_name="llama3-8b-8192",
    model_name="llama-3.2-3b-preview",
    temperature=0.1,
    max_tokens=1000,

)
"""
# OR Use Ollama to run Locally
from langchain_community.llms import Ollama
llm = Ollama(
    model="llama3.1:8b", #llama3.2:latest",
    temperature=0,
    # other params...
)


In [18]:
#!pip install tf-keras
#!pip install ollama

In [34]:

"""

rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

"""


rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="ollama", # or google, openai, anthropic, llama2, ...
            config=dict(
                #model="llama3-8b-8192",
                model="llama3.1:8b",#"llama3.2:latest",
                base_url="http://localhost:11434"
            ),
        ),
        embedder=dict(
                provider="huggingface", # or openai, ollama, ...
                config=dict(
                    model="BAAI/bge-small-en-v1.5",
                   
                    # title="Embeddings",
                ),
            ),
    )
)



In [35]:
question = "How did self-attention mechanism evolve in large language models in pdf?"
rag_tool.run(question)

Using Tool: Search a PDF's content


'Relevant Content:\nThis makes it more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 22, 23, 19]. End-to-end memory networks are based on a recurrent attention mechanism instead of sequence- aligned recurrence and have been shown to perform well on simple-language question answering and language modeling tasks [28]. To the best of our knowledge, howev

In [36]:
## Using Tavily API Key for search , try with DuckDuck Go
web_search_tool = TavilySearchResults(k=3)

In [37]:
web_search_tool.run("How did self-attention mechanism evolve in large language models?")

[{'url': 'https://ieeexplore.ieee.org/document/10245906',
  'content': 'Transformers, originally introduced for machine translation, and built upon the Self-Attention mechanism, have undergone a remarkable evolution, establishing themselves as the bedrock of large language models (LLMs). Their unparalleled capacity to model intricate relationships and capture extensive dependencies within sequences has propelled their prominence. This article, presented in a popular science format, serves as an introduction to the transformer architecture, elucidating its [...] Opens in a new window Opens an external website Opens an external website in a new window\n\nThis website utilizes technologies such as cookies to enable essential site functionality, as well as for analytics, personalization, and targeted advertising. To learn more, view the following link:    Privacy Policy\n\nSelf-Attention and Transformers: Driving the Evolution of Large Language Models | IEEE Conference Publication | IEEE X

In [38]:
@tool
def simple_router_tool(question : str) -> str:
  """Routes queries to the appropriate resources"""
  if "pdf" in question.lower():
    return "vectorstore"
  else:
    return "websearch"



In [39]:
test_input = {"question": "What does the  say about machine learning in pdf?"}
simple_router_tool.run(test_input)

'vectorstore'

In [40]:
#Agent 1

Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

#Task 1
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    #tools=[simple_router_tool],
)

In [41]:
crew1 = Crew(
    agents = [Router_Agent],
    tasks = [router_task],
    verbose= True,
)
test_input = {"question": "What does the  say about machine learning in pdf?"}
result1 = crew1.kickoff(inputs=test_input)

2025-11-09 00:11:47,953 - 8410570944 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question What does the  say about machine learning in pdf?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore




In [42]:
print(result1)

vectorstore


In [43]:
web_search_tool = TavilySearchResults()
question = "How did self-attention mechanism evolve in large language models in pdf?"
web_search_tool(question)

[{'url': 'https://ieeexplore.ieee.org/document/10245906',
  'content': 'Transformers, originally introduced for machine translation, and built upon the Self-Attention mechanism, have undergone a remarkable evolution, establishing themselves as the bedrock of large language models (LLMs). Their unparalleled capacity to model intricate relationships and capture extensive dependencies within sequences has propelled their prominence. This article, presented in a popular science format, serves as an introduction to the transformer architecture, elucidating its [...] # Self-Attention and Transformers: Driving the Evolution of Large Language Models\n\nPDF "You do not have access to this PDF")\n\nQing Luo; Wei Zeng; Manni Chen; Gang Peng; Xiaofeng Yuan; Qiang Yin\n\nAll Authors)\n\n Alerts)\n\n  # Alerts\n\n  Manage Content Alerts\n\n  Add to Citation Alerts)\n\n Download PDF\n Download References)\n Request Permissions\n Save to\n Alerts\n\n## Abstract: [...] Opens in a new window Opens an ex

In [44]:
#Agent 2
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)




#Task 2
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
    tools=[rag_tool, web_search_tool],
)

In [46]:
# Crew for the System
simple_rag_crew = Crew(
    agents = [Router_Agent,Retriever_Agent],
    tasks = [router_task,retriever_task],
    verbose= True,
)


test_input = {"question" :  "How U-Net work?"}

#test_input = {"question" :  "Does dorthy have any friends?"}
result1 = simple_rag_crew.kickoff(inputs=test_input)

2025-11-09 00:18:07,721 - 8410570944 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How U-Net work?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question How U-Net work? with the help of the respective tool.Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'.Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'.


> Entering new CrewAgentExecutor chain...
To

In [ ]:
result1

'Agent stopped due to iteration limit or time limit.'

In [47]:
inputs ={"question":"How does self-attention mechanism help large language models in pdf?"}

#test_input = {"question" :  "Does dorthy have any friends?"}
result2 = simple_rag_crew.kickoff(inputs=inputs)
result2

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How does self-attention mechanism help large language models in pdf?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question How does self-attention mechanism help large language models in pdf? with the help of the respective tool.Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'.Use the rag_tool to retrieve information from the v

'In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence length is smaller than the representation dimensionality d, which is most often the case with sentence representations used by state-of-the-art models in machine translations. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations.\n\nThe Transformer model, based solely on attention mechanisms, has been shown to be superior in quality while being more parallelizable and requiring significantly less time to train.'